In [24]:
import json
import uproot
from tqdm import tqdm

# Input/output config
INPUT_JSON = "filelist_ZZ.json"
OUTPUT_JSON = "filelist_ZZ_validated.json"
DATASET_NAME = "ZZ_TuneCP5_13p6TeV_pythia8"

# Load input file list
with open(INPUT_JSON) as f:
    data = json.load(f)

file_map = data[DATASET_NAME]["files"]

# Normalize input: either dict or list
if isinstance(file_map, dict):
    file_items = list(file_map.items())  # dict of {file: object_path}
else:
    file_items = [(f, "Events") for f in file_map]  # assume "Events" tree

valid_file_items = []

print("🔍 Validating ROOT files...")

for path, objpath in tqdm(file_items):
    try:
        with uproot.open(f"{path}:{objpath}") as tree:
            n = tree.num_entries
            if n == 0:
                print(f"⚠️ Skipping (zero entries): {path}")
            else:
                valid_file_items.append((path, objpath))
    except Exception as e:
        print(f"❌ Skipping (unreadable): {path} — {e}")

# Rebuild structure to match input
if isinstance(file_map, dict):
    out_files = {f: obj for f, obj in valid_file_items}
else:
    out_files = [f for f, obj in valid_file_items]

validated = {DATASET_NAME: {"files": out_files}}

# Write output
with open(OUTPUT_JSON, "w") as f:
    json.dump(validated, f, indent=2)

print(f"\n✅ Wrote {len(valid_file_items)} valid files to {OUTPUT_JSON}")


🔍 Validating ROOT files...


100%|██████████| 100/100 [04:10<00:00,  2.51s/it]


✅ Wrote 100 valid files to filelist_ZZ_validated.json


In [12]:
from coffea.nanoevents import NanoEventsFactory
from coffea.nanoevents.schemas import NanoAODSchema

filename = "root://eosuser.cern.ch//eos/cms/store/group/phys_exotica/axol1tl/MC_ScoutingNano/ZZ_TuneCP5_13p6TeV_pythia8/ZZ_ScoutingNANOAOD_20250603/250603_204359/0000/output_ScoutingNANOAOD_100.root"

# Explicitly pass the tree name in a dict
events = NanoEventsFactory.from_root({filename: "Events"}, schemaclass=NanoAODSchema).events()

# To get length and inspect content, compute it
import dask_awkward as dak
print("Number of events:", dak.num(events, axis=0).compute())
print(events[0])



Number of events: 39150
dask.awkward<getitem, type=Record>


In [13]:
import json

with open("filelist_ZZ.json") as f:
    dataset = json.load(f)

print(json.dumps(dataset, indent=4))
print(type(dataset["ZZ_TuneCP5_13p6TeV_pythia8"]["files"]))
print(dataset["ZZ_TuneCP5_13p6TeV_pythia8"]["files"].keys())  # should print filenames


{
    "ZZ_TuneCP5_13p6TeV_pythia8": {
        "files": {
            "root://eosuser.cern.ch//eos/cms/store/group/phys_exotica/axol1tl/MC_ScoutingNano/ZZ_TuneCP5_13p6TeV_pythia8/ZZ_ScoutingNANOAOD_20250603/250603_204359/0000/output_ScoutingNANOAOD_1.root": "Events",
            "root://eosuser.cern.ch//eos/cms/store/group/phys_exotica/axol1tl/MC_ScoutingNano/ZZ_TuneCP5_13p6TeV_pythia8/ZZ_ScoutingNANOAOD_20250603/250603_204359/0000/output_ScoutingNANOAOD_10.root": "Events",
            "root://eosuser.cern.ch//eos/cms/store/group/phys_exotica/axol1tl/MC_ScoutingNano/ZZ_TuneCP5_13p6TeV_pythia8/ZZ_ScoutingNANOAOD_20250603/250603_204359/0000/output_ScoutingNANOAOD_100.root": "Events",
            "root://eosuser.cern.ch//eos/cms/store/group/phys_exotica/axol1tl/MC_ScoutingNano/ZZ_TuneCP5_13p6TeV_pythia8/ZZ_ScoutingNANOAOD_20250603/250603_204359/0000/output_ScoutingNANOAOD_11.root": "Events",
            "root://eosuser.cern.ch//eos/cms/store/group/phys_exotica/axol1tl/MC_ScoutingNano/ZZ_Tu

In [15]:
import uproot

file_url = "root://xcache//store/group/phys_exotica/axol1tl/Data_ScoutingNano_withAXOscore/2024I/output_1298_v2_run386814.root"
with uproot.open(file_url) as f:
    print(f.keys())


['Events;9', 'Events;8', 'Runs;1', 'LuminosityBlocks;1']


In [16]:
import uproot

file_path = "/tmp/output_ScoutingNANOAOD_1.root"

try:
    with uproot.open(file_path) as f:
        print("Keys in the ROOT file:")
        print(f.keys())  # List all keys (trees, folders, etc.)
        
        if "Events" in f:
            events = f["Events"]
            print(f"Number of entries in 'Events' tree: {events.num_entries}")
        else:
            print("No 'Events' tree found in the file.")
except Exception as e:
    print("Failed to open file:", e)


Keys in the ROOT file:
['tag;1', 'Events;1', 'LuminosityBlocks;1', 'Runs;1', 'MetaData;1', 'ParameterSets;1']
Number of entries in 'Events' tree: 17400


In [17]:
import uproot

remote_file = "root://eosuser.cern.ch:1094//eos/cms/store/group/phys_exotica/axol1tl/MC_ScoutingNano/ZZ_TuneCP5_13p6TeV_pythia8/ZZ_ScoutingNANOAOD_20250603/250603_204359/0000/output_ScoutingNANOAOD_1.root"

try:
    with uproot.open(remote_file) as f:
        print("Keys in remote ROOT file:")
        print(f.keys())
        events = f["Events"]
        print(f"Number of entries in 'Events' tree: {events.num_entries}")
except Exception as e:
    print("Failed to open remote file:", e)


Keys in remote ROOT file:
['tag;1', 'Events;1', 'LuminosityBlocks;1', 'Runs;1', 'MetaData;1', 'ParameterSets;1']
Number of entries in 'Events' tree: 17400
